In [1]:
%load_ext autoreload
%autoreload 2

import numpy as np
import pandas as pd
import torch
import os
import sys
sys.path.append("../")
import plotly.express as px

In [2]:

data_dir = "/data/cb/scratch/datasets/atlas_dynamics_labels"
data_config_dir = "../configs/data"

In [3]:
import yaml
from pathlib import Path

# Load YAML config
with open(data_config_dir+"/atlas_config.yaml", "r") as file:
    config = yaml.safe_load(file)
config

{'repo_dir': '/data/cb/mihirb14/projects/DynaProt',
 'data_dir': '/data/cb/scratch/datasets/atlas_dynamics_labels',
 'protein_chains_path': '/data/cb/mihirb14/projects/dynaprot/dynaprot/data/preprocessing/protein_lists/atlas_chains.npy',
 'split': 'all',
 'max_num_residues': 512}

In [4]:
config["split"] = "train"

In [ ]:
from dynaprot.data.datasets import DynaProtDataset, OpenFoldBatchCollator

dataset = DynaProtDataset(config)
print(len(dataset))
dataloader = torch.utils.data.DataLoader(
        dataset,
        batch_size=1,
        collate_fn=OpenFoldBatchCollator(),
        num_workers=12,
        shuffle=False,
    )

batch_prots = next(iter(dataloader))
print(len(dataloader))
for k in batch_prots.keys():
    # print(k,batch_prots[k])
    print(f"{k}: {batch_prots[k].shape}")

1266


1266
aatype: torch.Size([1, 512, 21])
residue_index: torch.Size([1, 512])
all_atom_positions: torch.Size([1, 512, 37, 3])
all_atom_mask: torch.Size([1, 512, 37])
frames: torch.Size([1, 512, 4, 4])
dynamics_means: torch.Size([1, 512, 3])
dynamics_covars: torch.Size([1, 512, 3, 3])
resi_pad_mask: torch.Size([1, 512])


: 

In [14]:
split = "val"
np.save(f"../dynaprot/data/preprocessing/protein_lists/atlas_chains_{split}.npy",pd.read_csv(f"splits/atlas_{split}.csv").name.to_numpy().astype(str))

In [11]:
np.load(f"../dynaprot/data/preprocessing/protein_lists/atlas_chains.npy")

array(['2pu9_A', '1kop_A', '4p7x_A', ..., '1bq8_A', '1z1l_A', '3agn_A'],
      dtype='<U6')

In [ ]:
batch_prots["dynamics_covars"][batch_prots["resi_pad_mask"].bool()].shape

In [7]:
# batch_prots["dynamics_covars"][batch_prots["resi_pad_mask"].bool()]

In [5]:
prot = torch.load("/data/cb/scratch/datasets/atlas_dynamics_labels/3nke_B/3nke_B.pt")
means = prot["dynamics_means"]
covars = prot["dynamics_covars"]

In [7]:
from dynaprot.evaluation.visualizer import plot_3d_gaussian_ellipsoids,plot_3d_gaussian_comparison

plot_3d_gaussian_ellipsoids(means.numpy()[:20], covars.numpy()[:20], num_points=50, save_path = "./3nke_B_ellipsoids.html")


In [9]:
import os
dir = set(os.listdir("/data/cb/scratch/datasets/atlas_dynamics_labels"))
lis = set(list(np.load("../dynaprot/data/preprocessing/protein_lists/atlas_chains.npy")))
lis==dir


True

In [12]:
"../dynaprot/data/preprocessing/protein_lists/atlas_chains.npy"[:-4]

'../dynaprot/data/preprocessing/protein_lists/atlas_chains'

In [10]:
%pwd

'/data/cb/mihirb14/projects/dynaprot/experiments'

In [ ]:
np.load("/data/cb/scratch/datasets/pdb_npz/c9/2c9i_D.npz",allow_pickle=True)["aatype"].argmax(axis=-1).shape

In [ ]:
means = []
vars = []
for i in range(len(dataset)):
    means.append(dataset[i]["dynamics_covars"].mean())
    vars.append(dataset[i]["dynamics_covars"].var())
    


In [ ]:
px.histogram(x=vars)

In [ ]:
feats["all_atom_positions"].shape

In [22]:
import os
import numpy as np
import pandas as pd

mmcif_path = "/data/cb/scratch/datasets/pdb_mmcif"
npz_path = "/data/cb/scratch/datasets/pdb_npz"
chain_path = "/data/cb/scratch/datasets/pdb_chains"

pdb_proteins = pd.read_csv(os.path.join(npz_path,"pdb_chains.csv"))["name"].str.split("_").str[0].astype(str)
np.save("/data/cb/mihirb14/projects/dynaprot/dynaprot/data/preprocessing/protein_lists/pdb_chains.npy",pdb_proteins)

In [27]:
atlas_proteins = np.unique(np.array(os.listdir("/data/cb/scratch/datasets/atlas_dynamics_labels/")))
np.save("/data/cb/mihirb14/projects/dynaprot/dynaprot/data/preprocessing/protein_lists/atlas_chains.npy",atlas_proteins)

2c9i_D, 2hqk_A